In [3]:
import pandas

data = pandas.read_csv('data.csv')

In [20]:
data_set, data_submission = data[data['shot_made_flag'].notnull()], data[data['shot_made_flag'].isnull()]

In [21]:
print len(data), len(data_set), len(data_submission)

30697 25697 5000


In [22]:
data_set

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5
5,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,9,3,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,6
6,Layup Shot,Layup,251,20000012,34.0443,0,0,-118.2698,8,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,7
8,Jump Shot,Jump Shot,265,20000012,33.9363,-65,108,-118.3348,6,3,...,2PT Field Goal,Left Side(L),In The Paint (Non-RA),8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,9
9,Running Jump Shot,Jump Shot,294,20000012,33.9193,-33,125,-118.3028,3,3,...,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,10
10,Jump Shot,Jump Shot,309,20000012,33.8063,-94,238,-118.3638,1,3,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,11
11,Jump Shot,Jump Shot,4,20000019,33.9173,121,127,-118.1488,11,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,12


In [23]:
# split to train-test
from sklearn.cross_validation import train_test_split

data_train, data_test = train_test_split(data_set, test_size = 0.3, random_state = 42)

In [68]:
print type(data_train)

<class 'pandas.core.frame.DataFrame'>


In [28]:
cat_cols = ['action_type', 'combined_shot_type', 'period', 'playoffs', 'season', 
            'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 
            'team_name', 'matchup', 'opponent']

In [44]:
data_train_cat, data_test_cat = data_train[cat_cols], data_test[cat_cols]

In [81]:
y_train, y_test = data_train['shot_made_flag'].values, data_test['shot_made_flag'].values

In [80]:
print y_train[:10]

[ 0.  0.  0.  0.  0.  1.  1.  0.  1.  1.]


In [50]:
x_data_train_cat, x_data_test_cat = data_train_cat.T.to_dict().values(), data_test_cat.T.to_dict().values()

In [51]:
print len(x_data_train_cat)

17987


In [82]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer(sparse = False)
x_data_train_cat_vect = vectorizer.fit_transform(x_data_train_cat)
x_data_test_cat_vect = vectorizer.transform(x_data_test_cat)

In [95]:
y_train.shape

(17987,)

In [96]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, n_jobs = 4)
clf.fit(x_data_train_cat_vect, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [97]:
predicted = clf.predict(x_data_test_cat_vect)

In [103]:
clf.score(x_data_test_cat_vect, y_test)

0.51271076523994807

In [100]:
print y_test

[ 0.  1.  0. ...,  0.  0.  1.]


In [102]:
from sklearn.metrics import accuracy_score

print accuracy_score(y_test.astype('int32'), predicted.astype('int32'))

0.51271076524


In [91]:
print len(y_train)

17987


In [65]:
import numpy
print numpy.sum(x_data_train_cat_vect, axis = 0)

[  6.70000000e+01   4.30000000e+01   4.00000000e+00   2.00000000e+00
   1.95000000e+02   3.80000000e+01   4.40000000e+01   1.00000000e+00
   2.00000000e+00   1.00000000e+01   1.70000000e+01   1.16300000e+03
   5.60000000e+01   3.40000000e+01   1.55000000e+02   1.80000000e+01
   6.07000000e+02   1.90000000e+01   2.30000000e+01   6.80000000e+01
   8.00000000e+00   2.00000000e+00   5.30000000e+01   2.12000000e+02
   1.30000000e+01   1.10140000e+04   1.51700000e+03   6.00000000e+00
   2.98000000e+02   3.00000000e+00   6.00000000e+00   1.00000000e+00
   4.10000000e+01   2.25000000e+02   1.10000000e+01   3.00000000e+01
   1.30000000e+01   4.00000000e+00   2.00000000e+00   2.80000000e+01
   5.39000000e+02   3.60000000e+01   3.00000000e+00   5.00000000e+00
   1.00000000e+00   1.00000000e+00   2.49000000e+02   7.00000000e+01
   1.00000000e+00   1.05000000e+02   4.10000000e+01   2.68000000e+02
   2.00000000e+00   5.00000000e+00   6.08000000e+02   8.10000000e+01
   7.78000000e+02   9.60000000e+01

In [66]:
print y_test

24546    0
26431    1
25051    0
26209    0
14336    0
7930     0
18521    0
27034    0
19191    1
17484    0
6411     0
15445    0
21794    1
17976    0
9842     1
17457    0
9997     0
6903     0
9512     1
27519    1
27337    0
23104    0
5093     1
17031    0
15506    0
10194    1
9069     0
15415    0
24501    1
1597     0
        ..
7506     1
22286    1
13671    0
17888    0
23689    1
23516    0
15754    0
1224     1
1066     0
24403    1
24807    0
9115     0
18475    1
7217     0
26489    0
10943    0
7918     0
9255     0
11824    0
7998     1
15741    1
8654     1
5883     1
28522    0
11437    1
9004     0
4756     1
28537    0
12717    0
20818    1
Name: shot_made_flag, dtype: float64
